# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt
# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tazovsky,67.4667,78.7000,-6.44,86,100,9.02,RU,2024-11-26
1,1,labytnangi,66.6572,66.4183,-5.42,90,100,10.14,RU,2024-11-26
2,2,wailua homesteads,22.0669,-159.3780,25.44,74,0,7.72,US,2024-11-26
3,3,vilyuysk,63.7553,121.6247,-20.06,91,100,3.65,RU,2024-11-26
4,4,port-aux-francais,-49.3500,70.2167,3.24,77,56,15.64,TF,2024-11-26


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=650,
    frame_height=500,
    size="Humidity",
    color="City"
    )

# Display the map
map_plot




:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Max Temp']<=12)\
                            &(city_data_df['Max Temp']>=0)\
                            &(city_data_df['Cloudiness']<=90)\
                            &(city_data_df['Country']!='US')\
                            &(city_data_df['Country']!='AU')\
                            &(city_data_df['Country']!='IN')\
                            &(city_data_df['Country']!='RU')\
                            &(city_data_df['Country']!='CN')\
                            &(city_data_df['Humidity']<70),:]
print(len(ideal_df))

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df.head()

24


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,punta arenas,-53.1500,-70.9167,10.06,66,20,4.12,CL,2024-11-26
43,43,iwanai,42.9744,140.5089,7.81,56,65,3.25,JP,2024-11-26
46,46,sastobe,42.5500,70.0000,9.79,33,90,2.54,KZ,2024-11-26
48,48,aswan,24.0934,32.9070,11.61,58,0,2.57,EG,2024-11-26
103,103,ushuaia,-54.8000,-68.3000,8.81,61,20,9.26,AR,2024-11-26


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
27,punta arenas,CL,-53.1500,-70.9167,66,
43,iwanai,JP,42.9744,140.5089,56,
46,sastobe,KZ,42.5500,70.0000,33,
48,aswan,EG,24.0934,32.9070,58,
103,ushuaia,AR,-54.8000,-68.3000,61,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
     # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
#         # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
#     # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# # Display sample data
hotel_df.head()

Starting hotel search
punta arenas - nearest hotel: Hotel Lacolet
iwanai - nearest hotel: ホチルいのう
sastobe - nearest hotel: No hotel found
aswan - nearest hotel: Oscar Hotel
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
rio gallegos - nearest hotel: Amérian Río Gallegos Apart & Suites
edirne - nearest hotel: westdoor hotel
yoichi - nearest hotel: エーヴランド ホテル
al jawf - nearest hotel: No hotel found
kristiansund - nearest hotel: Comfort Hotel Fosna
el calafate - nearest hotel: Hostería Las Avutardas
maych'ew - nearest hotel: Tadele Hotel
kandahar - nearest hotel: No hotel found
llanes - nearest hotel: Los Molinos
leninskoye - nearest hotel: No hotel found
zapala - nearest hotel: Apart Solar de Piedra
bakanas - nearest hotel: Карой
kujang-up - nearest hotel: No hotel found
semnan - nearest hotel: مرکز آموزشی رفاهی فرهنگیان - خانه معلم
ar rutbah - nearest hotel: No hotel found
aliveri - nearest hotel: Olympic Star
hammerfest - nearest hotel: Thon Hotel Hammerfest
novyy turtkul' - near

,City,Country,Lat,Lng,Humidity,Hotel Name
27,punta arenas,CL,-53.1500,-70.9167,66,Hotel Lacolet
43,iwanai,JP,42.9744,140.5089,56,ホチルいのう
46,sastobe,KZ,42.5500,70.0000,33,No hotel found
48,aswan,EG,24.0934,32.9070,58,Oscar Hotel
103,ushuaia,AR,-54.8000,-68.3000,61,Apart Hotel Aires del Beagle


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
plot_4 = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)

plot_4


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)